# Save predictions from the models trained with synthesized data to Pandas


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('donkeycar')
sys.path.append('src')

In [3]:
import os
# do we have the correct env loaded?
assert 'donkey2' == os.environ['CONDA_DEFAULT_ENV'], os.environ['CONDA_DEFAULT_ENV']

In [4]:
![ ! -d "mycar/" ] && \
donkey createcar --path mycar && \
cp config/myconfig.py mycar/

________             ______                   _________              
___  __ \_______________  /___________  __    __  ____/_____ ________
__  / / /  __ \_  __ \_  //_/  _ \_  / / /    _  /    _  __ `/_  ___/
_  /_/ // /_/ /  / / /  ,<  /  __/  /_/ /     / /___  / /_/ /_  /    
/_____/ \____//_/ /_//_/|_| \___/_\__, /      \____/  \__,_/ /_/     
                                 /____/                              

using donkey v4.3.5 ...
Creating car folder: mycar
making dir  mycar
Creating data & model folders.
making dir  mycar/models
making dir  mycar/data
making dir  mycar/logs
Copying car application template: complete
Copying car config defaults. Adjust these before starting your car.
Copying train script. Adjust these before starting your car.
Copying calibrate script. Adjust these before starting your car.
Copying my car config overrides
Donkey setup complete.
cp: config/myconfig.py: No such file or directory


In [13]:
import donkeycar as dk
cfg = dk.load_config(config_path='mycar/config.py')

# None means all the data is used.
# use a smaller size, like 200 for testing end to end.
TUBRECORD_SIZE = 10150

cfg.SEQUENCE_LENGTH = 3
cfg.WANDB_ENABLED = False
cfg.TRANSFORMATIONS = ['CROP']
cfg.ROI_CROP_TOP = 60
cfg.ROI_CROP_KEEP_SIZE = False

DIRECTION = 'CC'
N_FOLDS = 5
DATA_PATH = '../data'


loading config file: mycar/config.py
loading personal config over-rides from myconfig.py


In [14]:
from dataloader import load_every_second_record
fast_speed_data = load_every_second_record('slower-autodrive', cfg, DATA_PATH, TUBRECORD_SIZE)
len(fast_speed_data)


INFO:donkeycar.pipeline.types:Loading only the first tub from paths ['../data/slower-autodrive']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/synthesized/../data/slower-autodrive/catalog_20.catalog


10150

In [15]:
from dataloader import load_records

slow_speed_data = load_records('slower-autodrive', cfg, DATA_PATH, TUBRECORD_SIZE)
len(slow_speed_data)


INFO:donkeycar.pipeline.types:Loading only the first tub from paths ['../data/slower-autodrive']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/synthesized/../data/slower-autodrive/catalog_20.catalog


10150

In [16]:
from donkeycar.utils import get_model_by_type

model_type = '3d-modified-steering'
model = get_model_by_type(model_type, cfg)
model.interpreter.model.summary()

INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3


Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img_in (InputLayer)         [(None, 3, 60, 160, 3)]   0         
                                                                 
 conv3d_6 (Conv3D)           (None, 1, 20, 53, 16)     1312      
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 1, 10, 26, 16)    0         
 3D)                                                             
                                                                 
 conv3d_7 (Conv3D)           (None, 1, 10, 26, 32)     4640      
                                                                 
 max_pooling3d_5 (MaxPooling  (None, 1, 5, 13, 32)     0         
 3D)                                                             
                                                                 
 conv3d_8 (Conv3D)           (None, 1, 5, 13, 32)      9248  

In [17]:
from losses import get_ground_truth
fast_ground_truths = get_ground_truth(model, fast_speed_data)
slow_ground_truths = get_ground_truth(model, slow_speed_data)


seq size 3
len ground_truth 10150
seq size 3
len ground_truth 10150


In [18]:
from dataloader import get_fold_indices
from losses import get_predictions
from model_loader import load_model
import numpy as np


train_predictions = np.zeros(len(fast_speed_data))
test_predictions = np.zeros(len(fast_speed_data))
slow_predictions = np.zeros(len(slow_speed_data))

MODELS_PATH = '/Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-synth-auto-3d-5-folds'

for fold, (train_index, test_index) in enumerate(get_fold_indices(fast_speed_data), start=1):
    train_records = [fast_speed_data[i] for i in train_index]
    test_records = [fast_speed_data[i] for i in test_index]
    trained_model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, N_FOLDS, fold)
    fold_train_predictions = np.zeros(len(fast_speed_data))
    np.put(fold_train_predictions, train_index, get_predictions(trained_model, cfg, train_records))
    train_predictions += fold_train_predictions

    fold_test_predictions = get_predictions(trained_model, cfg, test_records)
    np.put(test_predictions, test_index, fold_test_predictions)

    fold_slow_predictions = get_predictions(trained_model, cfg, slow_speed_data)
    slow_predictions += fold_slow_predictions

train_predictions /= (N_FOLDS-1)
slow_predictions /= N_FOLDS

INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-synth-auto-3d-5-folds/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-synth-auto-3d-5-folds/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5


63/63 [==============================] - 59s 929ms/step
predictions shape (8063,)
17/17 [==============================] - 15s 849ms/step
predictions shape (2087,)
80/80 [==============================] - 70s 871ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-synth-auto-3d-5-folds/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-synth-auto-3d-5-folds/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5


predictions shape (10150,)
63/63 [==============================] - 19s 306ms/step
predictions shape (8063,)
17/17 [==============================] - 6s 333ms/step
predictions shape (2087,)
80/80 [==============================] - 25s 316ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-synth-auto-3d-5-folds/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-synth-auto-3d-5-folds/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5


...

predictions shape (10150,)
64/64 [==============================] - 22s 336ms/step
predictions shape (8158,)
16/16 [==============================] - 9s 568ms/step
predictions shape (1992,)
80/80 [==============================] - 37s 459ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1


predictions shape (10150,)


INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-synth-auto-3d-5-folds/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-synth-auto-3d-5-folds/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5


64/64 [==============================] - 24s 371ms/step
predictions shape (8158,)
16/16 [==============================] - 6s 391ms/step
predictions shape (1992,)
80/80 [==============================] - 26s 320ms/step
predictions shape (10150,)


In [19]:
train_predictions.mean(), test_predictions.mean(), slow_predictions.mean()


(-0.25410469849451717, -0.25303979723819814, -0.25240241868302155)

In [20]:
train_predictions.shape, test_predictions.shape, slow_predictions.shape


((10150,), (10150,), (10150,))

In [21]:
np.save('nighttime-fast-auto-3d-model-nighttime-synth-fast-synth-auto-data-prediction-and-ground-truth.npy', np.array([train_predictions, test_predictions, fast_ground_truths]).T)
np.save('nighttime-fast-auto-3d-model-nighttime-synth-slow-auto-data-prediction-and-ground-truth.npy', np.array([slow_predictions, slow_ground_truths]).T)


In [22]:

train_predictions = np.zeros(len(slow_speed_data))
test_predictions = np.zeros(len(slow_speed_data))
fast_predictions = np.zeros(len(fast_speed_data))

MODELS_PATH = '/Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-auto-3d-5-folds-with-half-slow-speed-data'

for fold, (train_index, test_index) in enumerate(get_fold_indices(slow_speed_data), start=1):
    train_records = [slow_speed_data[i] for i in train_index]
    test_records = [slow_speed_data[i] for i in test_index]
    trained_model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, N_FOLDS, fold)
    fold_train_predictions = np.zeros(len(slow_speed_data))
    np.put(fold_train_predictions, train_index, get_predictions(trained_model, cfg, train_records))
    train_predictions += fold_train_predictions

    fold_test_predictions = get_predictions(trained_model, cfg, test_records)
    np.put(test_predictions, test_index, fold_test_predictions)

    fold_fast_predictions = get_predictions(trained_model, cfg, fast_speed_data)
    fast_predictions += fold_fast_predictions

train_predictions /= (N_FOLDS-1)
fast_predictions /= N_FOLDS

INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-auto-3d-5-folds-with-half-slow-speed-data/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-auto-3d-5-folds-with-half-slow-speed-data/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5


63/63 [==============================] - 14s 221ms/step
predictions shape (8063,)
17/17 [==============================] - 4s 234ms/step
predictions shape (2087,)
80/80 [==============================] - 21s 256ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-auto-3d-5-folds-with-half-slow-speed-data/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-auto-3d-5-folds-with-half-slow-speed-data/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5


predictions shape (10150,)
63/63 [==============================] - 17s 261ms/step
predictions shape (8063,)
17/17 [==============================] - 5s 270ms/step
predictions shape (2087,)
80/80 [==============================] - 20s 251ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-auto-3d-5-folds-with-half-slow-speed-data/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-auto-3d-5-folds-with-half-slow-speed-data/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5


predictions shape (10150,)
64/64 [==============================] - 17s 259ms/step
predictions shape (8158,)
16/16 [==============================] - 5s 321ms/step
predictions shape (1992,)
80/80 [==============================] - 23s 292ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-auto-3d-5-folds-with-half-slow-speed-data/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-auto-3d-5-folds-with-half-slow-speed-data/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5


predictions shape (10150,)
64/64 [==============================] - 17s 268ms/step
predictions shape (8158,)
16/16 [==============================] - 4s 273ms/step
predictions shape (1992,)
80/80 [==============================] - 21s 258ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-auto-3d-5-folds-with-half-slow-speed-data/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-auto-3d-5-folds-with-half-slow-speed-data/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5


predictions shape (10150,)
64/64 [==============================] - 14s 221ms/step
predictions shape (8158,)
16/16 [==============================] - 4s 232ms/step
predictions shape (1992,)
80/80 [==============================] - 19s 231ms/step
predictions shape (10150,)


In [23]:
train_predictions.mean(), test_predictions.mean(), fast_predictions.mean()


(-0.25744864634380404, -0.25722181979456704, -0.2564220145786264)

In [24]:
train_predictions.shape, test_predictions.shape, fast_predictions.shape


((10150,), (10150,), (10150,))

In [25]:
np.save('nighttime-slow-auto-3d-model-nighttime-slow-auto-data-prediction-and-ground-truth.npy', np.array([train_predictions, test_predictions, slow_ground_truths]).T)
np.save('nighttime-slow-auto-3d-model-nighttime-fast-synth-auto-data-prediction-and-ground-truth.npy', np.array([fast_predictions, fast_ground_truths]).T)
